# 对零售业的客户数据进行RFM分析和用户划分

使用数据用包含客户id、交易时间、收款金额三个维度。

In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 读取数据

df = pd.read_csv('../input/retailtransactiondata/Retail_Data_Transactions.csv', parse_dates=['trans_date'])

In [ ]:
# 查看前三条数据
df.head(3)

每条数据由单次交易产生

In [ ]:
df.info()

确定最早和最晚交易记录

In [ ]:
# 最早和最晚交易记录

print(df['trans_date'].min(), df['trans_date'].max())

假设分析时间为2015年4月1日，并以此计算最近一次消费（Recency）

In [ ]:
sd = dt.datetime(2015,4,1)
df['hist']=sd - df['trans_date']
df.head(3)

In [ ]:
# 提取天数的值

df['hist'].astype('timedelta64[D]')
df['hist']=df['hist'] / np.timedelta64(1, 'D')
df.head(3)

只考虑两年之内的交易，去掉两年前的数据

In [ ]:
df=df[df['hist'] < 730]
df.info()

统计每个客户的最近消费时间（Recency）、消费次数（Frequency）、消费总金额（Monetary Value）

In [ ]:
rfmTable = df.groupby('customer_id').agg({'hist': lambda x:x.min(),         # Recency
                                        'customer_id': lambda x: len(x),    # Frequency
                                        'tran_amount': lambda x: x.sum()})  # Monetary Value

rfmTable.rename(columns={'hist': 'recency', 
                         'customer_id': 'frequency', 
                         'tran_amount': 'monetary_value'}, inplace=True)

In [ ]:
rfmTable.head()

交叉比对客户'CS1112'的交易信息和rfm数据. 

In [ ]:
df[df['customer_id']=='CS1112']

RFM分析通常在消费时间（R）、消费次数（F）、消费总金额（M）三个维度上将用户分别进行归类。    
这次分析按照四分位线将客户分为4类。

In [ ]:
# 查看RFM三个维度的四分线

quartiles = rfmTable.quantile(q=[0.25,0.50,0.75])
print(quartiles, type(quartiles))

将四分位线存进词典

In [ ]:
quartiles=quartiles.to_dict()
quartiles


在RFM三个维度中，最近消费时间（R）越小越好，消费次数（F）和消费总金额（M）越大越好。   
所以在分类计分的时候，R越低得分越高。

In [ ]:
# 计算最近消费时间（R）得分

def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# 消费次数（F）和消费总额（M）得分

def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1    
    

In [ ]:
rfmSeg = rfmTable
rfmSeg['R_Quartile'] = rfmSeg['recency'].apply(RClass, args=('recency',quartiles,))
rfmSeg['F_Quartile'] = rfmSeg['frequency'].apply(FMClass, args=('frequency',quartiles,))
rfmSeg['M_Quartile'] = rfmSeg['monetary_value'].apply(FMClass, args=('monetary_value',quartiles,))

对客户进行分类需要将三项评分结合到一起来看。最简单的一种办法就是将评分组合出一个三位数字。但是这样做的缺点就是产生出的分类组合太多。

In [ ]:
rfmSeg['RFMClass'] = rfmSeg.R_Quartile.map(str) \
                            + rfmSeg.F_Quartile.map(str) \
                            + rfmSeg.M_Quartile.map(str)

In [ ]:
rfmSeg.head()

In [ ]:
rfmSeg.sort_values(by=['RFMClass', 'monetary_value'], ascending=[True, False])

In [ ]:
rfmSeg.groupby('RFMClass').agg('monetary_value').mean()

另一个组合评分的方法就是对三项分数进行求和。这个方法产生的分组较少，但它的缺点是，这样的分组体现不了某些细分客户的特点，比如421和142会被分在同一组。

In [ ]:
rfmSeg['Total Score'] = rfmSeg['R_Quartile'] + rfmSeg['F_Quartile'] +rfmSeg['M_Quartile']
print(rfmSeg.head(), rfmSeg.info())

In [ ]:
# 每个分组的消费平均值

rfmSeg.groupby('Total Score').agg('monetary_value').mean()

用柱状图直观表示每个得分组别的RFM平均值。

In [ ]:
rfmSeg.groupby('Total Score').agg('monetary_value').mean().plot(kind='bar', colormap='Blues_r')

In [ ]:
rfmSeg.groupby('Total Score').agg('frequency').mean().plot(kind='bar', colormap='Blues_r')

In [ ]:
rfmSeg.groupby('Total Score').agg('recency').mean().plot(kind='bar', colormap='Blues_r')

可以较明显地看到，各组的RFM平均值与得分呈正相关。

最后检查各个分组客户对促销活动的反应。

In [ ]:
res = pd.read_csv('../input/retailtransactiondata/Retail_Data_Response.csv')
res.sort_values('customer_id', inplace=True)

print(res.head(), res.info())

In [ ]:
rfmSeg.reset_index(inplace=True)
rfmSeg.head()

In [ ]:
rfmSeg.sort_values('customer_id', inplace=True)
rfm2=pd.merge(rfmSeg, res, on='customer_id')

In [ ]:
rfm2.head()

In [ ]:
rfm2.info()

In [ ]:
ax=rfm2.groupby('Total Score').agg('response').mean().plot(kind='bar', colormap='copper_r')
ax.set_xlabel("Total Score")
ax.set_ylabel("Proportion of Responders")

柱状图反映出客户对促销活动的响应率与客户的RFM得分有很强的相关性。并在得分6和7之间呈现出一个脚大的差距。